In [504]:
# 데이터프레임에서 Contents 컬럼 Series 가져오기
import pandas as pd

df = pd.read_csv('./경제_20230705_202522_page100001_99991.csv')
contents = df['content']

# 판다스 댓글(comments)를 댓글 하나씩 가져와 정제(re)

import re

def text_clean(text):
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail제거
    text = re.sub(pattern, '', text)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
    text = re.sub(pattern, '', text)
    pattern = '[a-zA-Z0-9]'    # 숫자와 알파벳 제거
    text = re.sub(pattern, '', text)
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
    text = re.sub(pattern, '', text)
    pattern = '<[^>]*>'         # HTML 태그 제거
    text = re.sub(pattern, '', text)
    pattern = '[^\w\s]'         # 특수기호제거
    text = re.sub(pattern, '', text)
    pattern = '[\n]'            # \n제거
    text = re.sub(pattern, '', text)
    pattern = '[\t]'            # \n제거
    text = re.sub(pattern, '', text)
    return text  


In [505]:
# 댓글들(comments)에서 하나씩 가져와 reply라는 변수에 list로 저장

replies = []

for comment in contents:
  replies.append(text_clean(comment))

print(replies[:5])

['경총대한상의중기중앙회소공연 반발부작용 완화 방안 마련해야서울연합뉴스 신선미 김철선 기자  내년도 최저임금이 올해보다  오른 시간당 천원으로 결정된 것과 관련해 경제단체는 강하게 반발하면서 국민 경제에 대한 부작용 완화 방안을 강구해야 한다고 정부에 촉구했다    한국경영자총협회경총는 일 입장문을 통해 코로나 여파와 고물가고금리고환율 등 중고가 겹치면서 더 이상 버티기 힘든 중소영세기업과 소상공인들의 현실을 외면한 결정이라고 비판했다    경총은 또 최근 년간 물가보다 배 이상 빠르게 오른 최저임금 수준 한계에 이른 중소영세기업과 소상공인의 지불능력 법에 예시된 결정요인 최근의 복합경제위기까지 종합적으로 감안했을 때 이번 의 인상률은 동의하기 어렵다고 밝혔다    경총은 이어 한계에 다다른 일부 업종의 최저임금 수용성조차 감안하지 않은 금번 결정으로 업종별 구분 적용의 필요성은 더욱 뚜렷해졌다며 정부는 업종별 구분 적용을 위한 실질적 방안을 조속히 마련하고 내년 심의 시에는 반드시 최저임금 구분 적용이 시행되어야 할 것이라고 강조했다    최저임금위원회는 전날 제차 전원회의를 열어 논의를 진행했으나 경영계와 노동계가 이견을 좁히지 못하자 공익위원들이 제출한 중재안을 표결에 부쳐 자정께 내년도 최저임금을 결정했다    하지만 사용자위원 명은 최저임금 중재안에 반발하면서 표결을 앞두고 전원 퇴장했다최저임금 원 제시 표결 거부하고 퇴장하는 사용자 위원들세종연합뉴스 김주형 기자  공익위원이 내년도 최저임금 원을 제시한 가운데 일 오후 세종시 정부세종청사 최저임금위원회 회의실에서 표결을 거부하고 퇴장한 류기정 한국경영자총협회 전무를 비롯한 사용자 위원들이 승강기에 오르고 있다  대한상공회의소대한상의는 강석구 조사본부장 명의의 논평에서 이번 최저임금 인상이 고물가고금리고환율 등으로 어려움을 겪고 있는 기업 부담을 한층 가중시키지 않을까 우려된다고 지적했다    대한상의는 노동생산성 증가율을 뛰어넘는 최저임금 인상은 중소기업과 소상공인의 경영난을 가중시키고 소속 근로자의

In [506]:
from sklearn.model_selection import train_test_split

In [507]:
import codecs

positive = []
unknown = []
    
# pos = codecs.open("./pos_pol_word.txt", 'rb', encoding='UTF-8')

# while True:
#     line = pos.readline()
#     line = line.replace('\r\n', '')
#     positive.append([line,1.0])
#     posnegun.append([line,1.0])
#     if not line: break        
# pos.close()

# txt 파일 불러오기
with open('./SentiWord_Dict.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()

# 첫 번째 줄을 열 이름으로 지정
columns = lines[0].strip().split(',')

# 데이터 로드
data = pd.DataFrame([line.strip().split('\t') for line in lines[1:]], columns=columns)

# 데이터 확인
data

,text,sentiment
0,(-;,1
1,(;_;),-1
2,(^^),1
3,(^-^),1
4,(^^*,1
...,...,...
14850,갈등,-1
14851,의혹,-1
14852,내팽개치다,-2
14853,횡령,-2


In [515]:
data.to_csv('./SentiWord_Dict.csv', index_label=True)

In [518]:
import pandas as pd
from konlpy.tag import Okt

# 감성 어휘 사전 로드
sentiment_lexicon = pd.read_csv('./SentiWord_Dict.csv', encoding='utf-8')
sentiment_words = set(sentiment_lexicon['text'])

# 형태소 분석기 초기화
okt = Okt()

# 뉴스 데이터 로드
news_data = pd.read_csv('./경제_20230705_202522_page100001_99991.csv')  # 뉴스 데이터 파일 경로에 맞게 수정해주세요.

# 뉴스 텍스트 감성 분석
sentiments = []
for text in news_data['content']:  # 뉴스 텍스트가 있는 열의 이름에 맞게 수정해주세요.
    tokens = okt.morphs(text)
    sentiment_score = 0
    for token in tokens:
        if token in sentiment_words:
            sentiment_score += sentiment_lexicon[sentiment_lexicon['text'] == token]['sentiment'].values[0]
    sentiment = '긍정' if sentiment_score > 0 else '부정' if sentiment_score < 0 else '중립'
    sentiments.append(sentiment)

# 감성 결과 출력
for i, sentiment in enumerate(sentiments):
    print(f"뉴스: {news_data['title'][i]}")  # 뉴스 텍스트가 있는 열의 이름에 맞게 수정해주세요.
    print(f"감성 분류: {sentiment}")
    print()



뉴스: 최저임금 인상에 경제계 "현실외면한 결정…인상률 동의 어려워"
감성 분류: 부정

뉴스: 9620원…경제수장도 ‘6% 물가’ 본다는데 5% 오른 최저임금
감성 분류: 중립

뉴스: 尹 "한국전 참전에 고마움…경제 안보 협력 강화"
감성 분류: 긍정

뉴스: 尹, 나토 일정 마지막날 '경제 외교'...연기된 나토 총장 면담
감성 분류: 긍정

뉴스: 치솟는 가계대출 금리… 한 총리 “경제전쟁 상황”
감성 분류: 긍정

뉴스: 尹, 영국 총리 만나 `프레임워크` 채택…·경제·과학 등 협력키로
감성 분류: 긍정

뉴스: 윤 대통령 순방, 경제 성과는?…"방산, 원전에서 성과"
감성 분류: 긍정

뉴스: 한미일 대북 공조 재확인·경제 세일즈…尹, 안보 지평 넓혔다
감성 분류: 긍정

뉴스: 다시 열린 김포~하네다… “경제-문화 교류도 열리길”
감성 분류: 긍정

뉴스: 한은 "한국 경제, 중국 의존도 심화 돼"
감성 분류: 부정

뉴스: 尹경제외교, 방산서 첫 성과낸다…"폴란드와 심도있게 논의"
감성 분류: 긍정

뉴스: 韓총리 “경제전쟁이라고 할만큼 대내외 상황 급박”…신속대응 주문
감성 분류: 부정

뉴스: '국비 전문가' 이종화 신임 경제부시장 "신공항 국비 건설, 대구 요구할 자격 충분"
감성 분류: 긍정

뉴스: 尹대통령, 스페인 경제인들 만나…"아시아의 라틴, 한국에 투자"
감성 분류: 긍정

뉴스: 제12대 경남도의회 중점 처리분야 1순위 '경제·일자리'
감성 분류: 부정

뉴스: 스페인 경제인 만난 윤석열 대통령
감성 분류: 중립

뉴스: [르포] "경제 살릴 것" "계엄의 아들"…필리핀 '마르코스2' 기대반 우려반
감성 분류: 긍정

뉴스: 부동산 투기 혐의 송병기 전 울산 경제부시장 징역 7년 구형
감성 분류: 부정

뉴스: 윤석열 대통령, 스페인 경제인과 오찬 간담회 [뉴시스Pic]
감성 분류: 긍정

뉴스: 경제부처서 김앤장으로…재취업 공직자 최소 100명
감성 분류: 부정

뉴스: 韓中전문가 "양국 경제 상호 의존적, RCEP·콘텐츠로 